# 영화 리뷰 감성분석 경진대회

`-` NLP는 어려워

`-` 딥러닝을 이용한 자연어 처리 입문 강의를 보고 따라하며 익히자

`-` 참고: https://wikidocs.net/44249

## 패키지 import및 데이터 전치리

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [250]:
data = pd.read_csv('C:/Users/Jaesu/github_desktop/Dacon-Basic/영화-리뷰-감성분석/Data/train.csv')

In [251]:
data.head(10)

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0
5,6,내인생에 있어 손까락에 꼽히는 영화 원작을 능가한 리메이크작,1
6,7,성룡영화중 최악 .. 차라리 러시아워가 훨씬 나음,0
7,8,마음이 성숙하고 보면 다르게 보이는 영화.. 걸작이다.,1
8,9,"가슴 찡했던, 오래 기억에남을 영화!!",1
9,10,결말 기분나쁘다 공포매니아에게 이따위 결말을,0


`-` 긍정 리뷰는 label이 1, 부정 리뷰는 label이 0으로 되어있다

`-` 영어가 포함되어 있는 리뷰의 수를 구해보자

In [69]:
from string import ascii_lowercase, ascii_uppercase

In [70]:
ascii_lowercase, ascii_uppercase ## a부터 z까지, A부터 Z까지

('abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ')

In [74]:
data.loc[data['document'].apply(lambda x: True if len(set(x) & set(ascii_lowercase)) or len(set(x) & set(ascii_uppercase)) else False)].head(10)

,id,document,label
26,27,initiation을 한글로 이니테이션이랜다,0
66,67,부분부분 수OO전 스토리를 떠올리게 하는 신기함 이외에는.,0
93,94,이유없음과정없음설득력없음BGM도없ㅋ음ㅋ감독맘대로 만든것같은 불친절한영화,0
114,115,뿌리깊은나무가 커피면 대왕세종은 TOP야,1
131,132,나레이션도 좋았고 OST도 좋았고 많은 생각을 하게 해준 드라마였다,1
159,160,very very very good,1
209,210,보리울의 여름 O.S.T.노래 정말 좋아요!!^.^,1
249,250,어릴대 TV에서 자주보던 거다 ㅎㅎ,1
256,257,OMG... 스토리가 천재적이네... 이런 소름끼치는 전개 좋다,1
345,346,왜 재미없는지 nc리뷰나 원작을 보고오세요 퍼니플래닛에 있음,0


`-` 영어가 포함된 리뷰는 210개이다

`-` 근데 딱히 긍정/부정을 가를만한 영단어는 없는 것 같으니 제외하자

`-` 또한 data를 보면 물결포, 온점, 느낌표등이 있는데 유의미한 단어는 아니니 이 역시 제거하자

`-` 한글과 띄어쓰기를 제외한 나머지를 제거하면 된다

### 데이터 정제

In [252]:
## 한글과 공백을 제외하고 모두 제거
data['document'] = data['document'].apply(lambda str_: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', str_))

In [253]:
data[['document']].head(10) 

,document
0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐
1,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯
2,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요
3,아무래도 년도지난작품이라 지금보기는너무유치하다
4,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다
5,내인생에 있어 손까락에 꼽히는 영화 원작을 능가한 리메이크작
6,성룡영화중 최악 차라리 러시아워가 훨씬 나음
7,마음이 성숙하고 보면 다르게 보이는 영화 걸작이다
8,가슴 찡했던 오래 기억에남을 영화
9,결말 기분나쁘다 공포매니아에게 이따위 결말을


`-` 한글과 띄어쓰기를 제외한 나머지가 잘 제거되었다

`-` 영어 혹은 특수문자만 사용하여 리뷰를 작성했을 수 도 있다

`-` 그러한 리뷰는 결측치로 처리하자

In [254]:
data['document'].apply(lambda str_: str_.replace(' ', '')).apply(lambda str_: str_ if len(str_) > 0 else np.nan).isnull().sum()

10

`-` 결측치가 10개다 ---> 영어 혹은 특수문자만 사용한 리뷰가 10개 존재한다

`-` 빈칸만 있는 리뷰를 보고 긍정/부정을 맞출 수 없다

`-` 이를 제거해주자

In [255]:
data.drop(index = data.loc[data['document'].\
                  apply(lambda str_: str_.replace(' ', '')).\
                  apply(lambda str_: False if len(str_) > 0 else True),:].\
                  index.tolist(), inplace = True)

In [256]:
data.shape

(4990, 3)

`-` 또한 ㅋ, ㅎ, ㅠ, ㅜ 등과 같이 의미없이 반복되는 글자도 제거해주자

`-` 위의 글자는 없어도 긍정/부정을 판단하는데 영향을 주지 않는다

In [257]:
_w = 'ㅋㅎㅜㅠㅗㅏㅜㅗㄱㄴㄷㅡㅛㅉ'

In [258]:
for w_ in _w:
    data['document'] = data['document'].apply(lambda str_: str_.replace(w_, ''))

### 토큰화

`-` 이제 문장을 의미있는 단어들로 쪼개는 토큰화를 하자

`-` 토큰화 과정에서 불용어는 리뷰가 긍정인지 부정인지 판단하는데 유용하지 않으므로 제거하자

`-` 불용어는 정하기 나름인데 일단은 아래 단어만 불용어로 지정하자

In [259]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다',
             '영화', '진짜', '평점', '재미', '정말', '스토리', '시간', '내용', '감독', '그냥']

## EDA에서 확인한 긍정/부정 둘 다 등장하는 상위 10개 단어 추가

`-` Okt를 이용하여 토큰화를 하자

In [260]:
okt = Okt()

In [261]:
X_train = []
for sentence in tqdm(data['document']):
    tokenized_sentence = okt.morphs(sentence, stem = True) ## 토큰화, stem = True를 사용하면 일정 수준의 정규화를 수행
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] ## 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|█████████████████████████████████████████████████████████████████████████████| 4990/4990 [00:38<00:00, 130.97it/s]


`-` 이제 단어를 기계가 처리할 수 있도록 숫자로 변환해주자(정수 인코딩)

`-` 훈련 데이터에 대해서 단어 집합(vocaburary)을 만들자

### 정수 인코딩

In [263]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

`-` 단어 집합이 생성되는 동시에 각 단어에 고유한 정수가 부여됐다

In [271]:
len(tokenizer.word_index) ## 6575개의 단어를 사용한다

6575

`-` 각 정수는 전체 훈련 데이터에서 등장 빈도수가 높은 순서대로 부여되었기 때문에

`-` 높은 정수가 부여된 단어들은 등장 빈도수가 매우 낮다는 것을 의미한다

In [289]:
threshold = 3
total_cnt = len(tokenizer.word_index) ## 단어의 수
rare_cnt = 0  ## 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0  ## 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0  ## 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

## 단어와 빈도수의 쌍(pair)을 key와 value로 받는다
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    ## 단어의 등장 빈도수가 threshold보다 작으면
    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s' %(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어의 비율:', round(rare_cnt / total_cnt, 3))
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율:', round(rare_freq / total_freq, 3))

단어 집합(vocabulary)의 크기 : 6575
등장 빈도가 2번 이하인 희귀 단어의 수: 4606
단어 집합에서 희귀 단어의 비율: 0.701
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.15


`-` 전체 단어중 등장 빈도가 2번 이하인 단어의 비율은 70%에 달한다

`-` 하지만 훈련집합에서 등장 빈도로 차지하는 비중은 15% 정도이다

`-` 제거해도 괜찮을 것 같다

`-` 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한하자

In [290]:
## 전체 단어 개수 중 빈도수 2이하인 단어는 제거
## 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :', vocab_size)

단어 집합의 크기 : 1970


`-` 텍스트 시퀀스를 정수 시퀀스로 변환하자

In [292]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

`-` 각 샘플 내의 단어들은 각 단어에 대한 정수로 변환됐다

`-` 단어의 개수는 1970개로 제한되었으므로 0번 단어 ~ 1969번 단어까지만 사용 중이다

`-` 0번 단어는 패딩(서로 다른 배열의 길이를 맞춰줌)을 위한 토큰

In [297]:
y_train = np.array(data['label'])

### 빈 샘플 제거

`-` 정수 인코딩 과정에서 빈도수가 낮은 단어만으로 구성되었던 샘플들은 빈(empty) 샘플이 되었다